# Data Ingestion and Preparation

## Configuration

In [ ]:
# Check free memory available
%system free -m

In [ ]:
# Import the necessary libraries
# Basic python libraries
import os
import string
import random

# Numeric and date libraries
#import numpy as np
#from datetime import datetime

# Google cloud libraries
from google.cloud import bigquery
from google.cloud import storage

# Pandas and BigQuery
# os.system("pip install pandas-gbq")
import pandas_gbq as pdg
import pandas as pd

In [ ]:
# Current working directory
os.getcwd()

# Display all columns
pd.set_option('display.max_columns', None)

In [ ]:
# Set output data locations
raw_data = "../data/raw/"
interim_data = "../data/interim/"
processed_data = "../data/processed/"

## Data Loading

### NCCIS Data

In [ ]:
# Define the SQL query to retrieve the information from the prepared nccis table
# Activity codes 110, 120, 130, 140 and 150 which signify those who have not yet reached the compulsory school leaving age were excluded
# It also restrict data to young NEET with Age at the start of academic year to 16 and 17
nccis = """
SELECT *
FROM `yhcr-prd-phm-bia-core.CB_2166.wide_format_NEET_final`
"""

In [ ]:
# Execute the query and retrieve the result into a DataFrame
nccis_df = pdg.read_gbq(nccis, dialect='standard')

# Display the result
nccis_df

### Congenital Heart Diseases Data

In [ ]:
# Define the SQL query to retrieve the congenital heart diseases (chd) information from the primary healthcare data
chd = """
SELECT DISTINCT *
FROM `yhcr-prd-phm-bia-core.CB_MYSPACE_FO.CongenitalAnomaliesCHD`
"""

In [ ]:
# Execute the query and retrieve the result into a DataFrame
chd_df = pdg.read_gbq(chd, dialect='standard')

# Display the result
chd_df

## Data Cleaning

### NCCIS Data

In [ ]:
## Modify the nccis data to fit purpose
nccis_df.info()

In [ ]:
# Rename the specific column names to follow a good naming convention
column_mapping = {
    'majority_LSOA_15_age': 'lsoa_code',
    'AcademicYearAge15to16': 'academic_year_age_15',
    'LSOA_name': 'lsoa_name',
    'Bradford_YN': 'home_lsoa_in_bradford',
    'ever_NEET': 'ever_neet_status',
    'Persistent_NEET_YN_over_4months': 'persistent_neet_status',
    'Total_neet_months': 'total_neet_obs',
    'total_number_of_observations': 'total_observations',
    'percentage_time_neet': 'percentage_neet_obs',
    'NumberOfMonthsUnknown': 'unknown_observation'
}

# Rename the columns
nccis_df.rename(columns=column_mapping, inplace=True)

# Display the result
nccis_df.info()

In [ ]:
# Based on instances where a person is present multiple responses to some attributes
nccis_duplicates = nccis_df.duplicated().sum()

# Display the result
print(f"Number of duplicates: {nccis_duplicates}")

In [ ]:
# Save dataframe to CSV file
csv_filename = "nccis_cleaned.csv"

# Combine the path and filename
csv_filepath = os.path.join(interim_data, csv_filename)

# Save the DataFrame to CSV
nccis_df.to_csv(csv_filepath, index=False)
print(f"DataFrame saved to: {csv_filepath}")

## Feature Engineering

### LSOAs and their Geometry Attributes

In [ ]:
# Define the SQL query to lookup for geometry attributes of all lsoa 
# Use Well-Known Text (WKT) to text represent the geometry of the lsoas
lsoa_general="""
SELECT
    a.LSOA_code AS lsoa_code,
    ST_X(a.lat_long) AS longitude,
    ST_Y(a.lat_long) AS latitude,
    ST_AsText(a.geometry) AS geometry,
    b.WD15CD AS ward_code,
    b.WD15NM AS ward_name,
    b.LAD15CD AS local_authority_code,
    b.LAD15NM AS local_authority_name,
    b.FID
FROM `yhcr-prd-phm-bia-core.CB_LOOKUPS.tbl_lsoa_boundaries` a
JOIN `yhcr-prd-phm-bia-core.CB_LOOKUPS.tbl_LSOA_to_Ward` b
ON a.LSOA_code = b.LSOA11CD
"""

In [ ]:
# Execute the query and retrieve the result into a DataFrame
lsoa_general_df = pdg.read_gbq(lsoa_general, dialect='standard')

# Display the result
lsoa_general_df

### Schools

In [ ]:
# Define the SQL query to load the data of schools in bradford
schools="""
SELECT
    LA__code_ AS la_code,
    EstablishmentNumber AS estab_number,
    EstablishmentName AS estab_name,
    EstablishmentStatus__name_ AS estab_status,
    LSOA__name_ AS school_lsoa_name,
    LSOA__code_ AS school_lsoa_code,
    OpenDate AS school_open_date,
    CloseDate AS school_close_date
FROM 
    `yhcr-prd-phm-bia-core.CB_MYSPACE_FO.BradfordAllSchools`
WHERE 
    EstablishmentNumber IS NOT NULL 
    #AND (CloseDate IS NULL OR CloseDate >= '2010-08-31')
    #AND (OpenDate IS NULL OR OpenDate <= '2018-09-01')
"""

In [ ]:
# Execute the query and retrieve the result into a DataFrame
schools_df = pdg.read_gbq(schools, dialect='standard')

# Display the result
schools_df

In [ ]:
# Create a mapping dictionary to anonymize school names
school_mapping = {}
for estab_name in schools_df['estab_name'].unique():
    code = ''.join(random.choices(string.ascii_uppercase + string.digits, k=3))
    school_mapping[estab_name] = code
    
# Replace school names with codes in schools_df
schools_df['school_code'] = schools_df['estab_name'].map(school_mapping)

In [ ]:
# Drop the 'estab_name' column from schools_df
schools_df.drop(columns=['estab_name'], inplace=True)

In [ ]:
# Display the result
schools_df

### Other Covariate Features

In [ ]:
# Define the SQL query to retrieve the information from the other covariates table
other_covariates="""
SELECT *
FROM `yhcr-prd-phm-bia-core.CB_MYSPACE_FO.OtherCovariates`
"""

In [ ]:
# Execute the query and retrieve the result into a DataFrame
other_covariates_df = pdg.read_gbq(other_covariates, dialect='standard')

# Display the result
other_covariates_df

In [ ]:
# Get unique values of the 'ethnicity' column along with their counts
ethnic_counts = other_covariates_df['ethnicity'].value_counts()

# Display the result
ethnic_counts

In [ ]:
# Mapping the 'ethnicity' column with the desired output
ethnicity_mapping = {
    'White: English or Welsh or Scottish or Northern Irish or British - England and Wales ethnic category 2011 census': 'White: English or Welsh or Scottish or Northern Irish or British',
    'Unknown/Refuse to say': 'Unknown/Refuse to say',
    'Asian or Asian British: Pakistani - England and Wales ethnic category 2011 census': 'Asian or Asian British: Pakistani',
    'White:Any other White background': 'White: Any other White background',
    'Asian or Asian British: Bangladeshi - England and Wales ethnic category 2011 census': 'Asian or Asian British: Bangladeshi',
    'Asian or Asian British: any other Asian background - England and Wales ethnic category 2011 census': 'Asian or Asian British: any other Asian background',
    'Asian or Asian British: Indian - England and Wales ethnic category 2011 census': 'Asian or Asian British: Indian',
    'Other ethnic group: any other ethnic group - England and Wales ethnic category 2011 census': 'Other ethnic group: any other ethnic group',
    'Black or African or Caribbean or Black British: African - England and Wales ethnic category 2011 census': 'Black or African or Caribbean or Black British: African',
    'Mixed multiple ethnic groups: any other Mixed or multiple ethnic background - England and Wales ethnic category 2011 census': 'Mixed multiple ethnic groups: any other Mixed or multiple ethnic background',    
    'Mixed multiple ethnic groups: White and Black Caribbean - England and Wales ethnic category 2011 census': 'Mixed multiple ethnic groups: White and Black Caribbean',
    'Black or African or Caribbean or Black British: Caribbean - England and Wales ethnic category 2011 census': 'Black or African or Caribbean or Black British: Caribbean',
    'Black or African or Caribbean or Black British: other Black or African or Caribbean background - England and Wales ethnic category 2011 census' : 'Black or African or Caribbean or Black British: other Black or African or Caribbean background',
    'Mixed multiple ethnic groups: White and Black African - England and Wales ethnic category 2011 census': 'Mixed multiple ethnic groups: White and Black African',
    'Asian or Asian British: Chinese - England and Wales ethnic category 2011 census': 'Asian or Asian British: Chinese',
    'White: Irish - England and Wales ethnic category 2011 census': 'White: Irish',
    'Other ethnic group: Arab - England and Wales ethnic category 2011 census': 'Other ethnic group: Arab',
    'White: Gypsy or Irish Traveller - England and Wales ethnic category 2011 census': 'White: Gypsy or Irish Traveller'                                               
}

# Replace using the mapped values
other_covariates_df['ethnicity'] = other_covariates_df['ethnicity'].replace(ethnicity_mapping)

# Produce the outcome to confirm the modification
ethnic_counts = other_covariates_df['ethnicity'].value_counts()

# Display the result
ethnic_counts

In [ ]:
# Ensure that the other_covariates_df are strictly  those in the nccis_df
other_covariates_df = pd.merge(nccis_df[['person_id']], other_covariates_df, on='person_id', how='left')

# Display the result
other_covariates_df

In [ ]:
# Save dataframe to CSV file
csv_filename = "other_covariates.csv"

# Combine the path and filename
csv_filepath = os.path.join(processed_data, csv_filename)

# Save the DataFrame to CSV
other_covariates_df.to_csv(csv_filepath, index=False)
print(f"DataFrame saved to: {csv_filepath}")

## NEET Summary Table

In [ ]:
# Create the neet_summary table by merging the nccis_df with other_covariates_df and school_df
neet_summary_df = pd.merge(nccis_df, lsoa_general_df, on='lsoa_code', how='left')
neet_summary_df = pd.merge(neet_summary_df, other_covariates_df, on='person_id', how='left')
neet_summary_df = pd.merge(neet_summary_df, schools_df, left_on='estab_at_age_15', right_on='estab_number', how='left')

# Display the merged DataFrame
neet_summary_df

In [ ]:
# Aggregate and arrange neet_summary_df accordingly
neet_summary_df = neet_summary_df.groupby('person_id').agg(
    date_of_birth=('date_of_birth', 'first'),
    majority_gender=('majority_gender', 'first'),
    academic_year_age_15=('academic_year_age_15', 'first'),
    academic_years=('academic_year', lambda x: ';'.join(sorted(set(x)))),
    ever_neet_status=('ever_neet_status', 'first'),
    persistent_neet_status=('persistent_neet_status', 'first'),
    total_neet_obs=('total_neet_obs', 'first'),
    unknown_observation=('unknown_observation', 'first'),
    total_observations=('total_observations', 'first'),
    custody_offender_yn=('custody_offender_yn', lambda x: any(x)),
    special_educational_needs=('special_educational_needs', lambda x: any(x)),
    learning_disability=('learning_disability', lambda x: any(x)),
    estab_at_age_15=('estab_at_age_15', 'first'),
    la_estab_at_age_15=('la_estab_at_age_15', 'first'),
    school_code=('school_code', 'first'),
    school_status=('estab_status', 'first'),
    school_lsoa_name=('school_lsoa_name', 'first'),
    school_open_date=('school_open_date', 'first'),
    school_close_date=('school_close_date', 'first'), 
    longitude=('longitude', 'first'),
    latitude=('latitude', 'first'),
    lat_long=('lat_long', 'first'),
    lsoa_code=('lsoa_code', 'first'),
    lsoa_name=('lsoa_name', 'first'),
    home_lsoa_in_bradford=('home_lsoa_in_bradford', 'first'),
    geometry=('geometry', 'first'),
    ward_code=('ward_code', 'first'),
    ward_name=('ward_name', 'first'),
    la_code=('local_authority_code', 'first'),
    la_name=('local_authority_name', 'first')  
).reset_index()

# Display the resulting DataFrame
neet_summary_df

In [ ]:
# Save dataframe to CSV file
csv_filename = "neet_summary.csv"

# Combine the path and filename
csv_filepath = os.path.join(processed_data, csv_filename)

# Save the DataFrame to CSV
neet_summary_df.to_csv(csv_filepath, index=False)
print(f"DataFrame saved to: {csv_filepath}")

## NEET Congenital Table

In [ ]:
# Create the neet_chd_df by merging the neet_summary_df with the chd_df
neet_chd_df = pd.merge(neet_summary_df, chd_df, on='person_id', how='left')

# Display the merged DataFrame
neet_chd_df

In [ ]:
# Create a new column 'chd_status' based on the condition
neet_chd_df['chd_status'] = neet_chd_df['snomedcode'].notnull()

# Display the resulted DataFrame
neet_chd_df

In [ ]:
# Aggregate the neet_chd_df to join the chd_name and obtain the first output for other columns
aggregation_rules = {
    'date_of_birth': 'first',
    'majority_gender': 'first',
    'academic_year_age_15': 'first',
    'academic_years': 'first',
    'ever_neet_status': 'first',
    'persistent_neet_status': 'first',
    'total_neet_obs': 'first',
    'unknown_observation': 'first',
    'custody_offender_yn': 'first',
    'special_educational_needs': 'first',
    'learning_disability': 'first',
    'estab_at_age_15': 'first',
    'la_estab_at_age_15': 'first',
    'school_code': 'first',
    'school_status': 'first',
    'school_lsoa_name': 'first',
    'school_open_date': 'first',
    'school_close_date': 'first',
    'longitude': 'first',
    'latitude': 'first',
    'lat_long': 'first',
    'lsoa_code': 'first',
    'lsoa_name': 'first',
    'home_lsoa_in_bradford': 'first',
    'geometry': 'first',
    'ward_code': 'first',
    'ward_name': 'first',
    'la_code': 'first',
    'la_name': 'first',
    'snomedcode': 'first',
    'chd_name': lambda x: ', '.join(x.astype(str).unique()),  # Combining non-null values of 'congenital_hd_name'
    'chd_status': 'first'
}

# Display the aggregated DataFrame
neet_chd_df = neet_chd_df.groupby('person_id').agg(aggregation_rules).reset_index()
neet_chd_df

In [ ]:
# Save dataframe to CSV file
csv_filename = "neet_chd.csv"

# Combine the path and filename
csv_filepath = os.path.join(processed_data, csv_filename)

# Save the DataFrame to CSV
neet_chd_df.to_csv(csv_filepath, index=False)
print(f"DataFrame saved to: {csv_filepath}")